# Description

* Merging raw reads with PEAR

## Setting variables

In [1]:
seqdir = '/home/backup_files/raw_reads/fullcyc2.Barnett.2018/fullCyc2_lib16/run1'
workdir = '/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib16'

library = 'fullCyc2_lib16'

readFile1 = 'read1.fq.gz'
readFile2 = 'read2.fq.gz'

# Init

In [2]:
import screed
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from mpld3 import enable_notebook
import screed
import pandas as pd
import os

In [3]:
%matplotlib inline  
%load_ext rpy2.ipython

In [4]:
%%R
library(ggplot2)
library(dplyr)

/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/home/sam/anaconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


### Check that your sequence files and directories do exist and are named as above

In [5]:
if not os.path.isdir(seqdir):
    print ('Sequence directory Path Does NOT exist!!')
else:
    %cd $seqdir

/home/backup_files/raw_reads/fullcyc2.Barnett.2018/fullCyc2_lib16/run1


In [6]:
!cd $seqdir; ls -thlc *fq.gz

-rw-rw-r-- 1 sam raw_reads 3.7G May 14 10:31 read2.fq.gz
-rw-rw-r-- 1 sam raw_reads 3.4G May 14 10:30 read1.fq.gz
-rw-rw-r-- 1 sam raw_reads 211M May 14 10:30 index_read2.fq.gz
-rw-rw-r-- 1 sam raw_reads 210M May 14 10:30 index_read1.fq.gz


In [7]:
if not os.path.isdir(workdir):
    os.makedirs(workdir)
%cd $workdir

/home/seq_data/fullCyc2/amplicon/515F-806R/final_dataset/library_QC/fullCyc2_lib16


## Uncompress the fastq files

In [8]:
output1 = os.path.join(workdir, "forward.fastq")
readFile1 = os.path.join(seqdir, readFile1)

!cd $workdir; \
    pigz -k -d -p 20 -c $readFile1 > $output1

In [9]:
output2 = os.path.join(workdir, "reverse.fastq")
readFile2 = os.path.join(seqdir, readFile2)

!cd $workdir; \
    pigz -k -d -p 20 -c $readFile2 > $output2

# Merging


In [10]:
!cd $workdir; \
    pear -m 600 -j 20 \
    -f forward.fastq \
    -r reverse.fastq \
    -o pear_merged-$library

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.10 [May 30, 2016]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: forward.fastq
Reverse reads file.................: reverse.fastq
PHRED..............................: 33
Using empirical frequencies........: YES
Statistical method.................: OES
Maximum assembly length............: 600
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads............................: 20

Allocating memory..................: 200,000,000 bytes
Computing empiric

# Making a screed db of merged reads

In [11]:
pear_merged_file = 'pear_merged-' + library + '.assembled.fastq'

os.chdir(workdir)
screed.read_fastq_sequences(pear_merged_file)

<ScreedDB, 'pear_merged-fullCyc2_lib16.assembled.fastq_screed'>

In [12]:
pear_merged_file += '_screed'
fqdb = screed.ScreedDB(pear_merged_file)

In [13]:
pear_merged_file

'pear_merged-fullCyc2_lib16.assembled.fastq_screed'

In [14]:
lengths = []
for read in fqdb.itervalues():
    lengths.append((len(read["sequence"])))

In [15]:
fig = plt.figure()
ax = fig.add_subplot(111)
h = ax.hist(np.array(lengths), bins=50)
xl = ax.set_xlabel("Sequence Length, nt")
yl = ax.set_ylabel("Count")
fig.set_size_inches((10,6))

In [ ]:
print ('Number of reads: {}'.format(len(lengths)))

Number of reads: 16120919


## Quality stats on merged reads


In [ ]:
def qualStats(sourceDir, fileName):
    outFile = fileName + '_qualStats'
    !cd $sourceDir; \
        fastx_quality_stats -i $fileName -o $outFile -Q 33
    return outFile
    
qualStatsRes = qualStats(workdir, 'pear_merged-' + library + '.assembled.fastq')

In [ ]:
%%R -i workdir -i qualStatsRes

setwd(workdir)

# reading in qual-stats files    
tbl.r12 = read.delim(qualStatsRes, sep='\t')
rownames(tbl.r12) = 1:nrow(tbl.r12)

In [20]:
%%R -w 800 -h 300
# smooth curve on median qual values
ggplot(tbl.r12, aes(x=column, y=med, ymin=Q1, ymax=Q3)) +
    geom_smooth(se=FALSE, method='auto') +
    geom_linerange(alpha=0.3) +
    labs(x='position', y='median quality score') +
    theme_bw() +
    theme( 
        text = element_text(size=16) 
        )

## Clean up
Remove temporary files made during this process. These are really big files that you no longer need. If you are worried about having to re-do something then you can skip this step and clean up manually at the very end.